## Trabajo práctico Integrador

**Alumno:** Torresetti Lisandro

**Padrón:** 99846

**Objetivo:**

Realizar un programa que permita determiar la pose de las piezas dispuestas en la zona de trabajo de manera que el robot pueda tomarlas.

In [ ]:
# Imports
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import PIL.ExifTags
import PIL.Image
import pprint
import re
%matplotlib inline

# Paths

BLOCKS_PATH = './imagenes/img_bloques'
BLOCKS_CHALLENGE_PATH = './imagenes/img_bloques_desafio'
CALIBRATION_SET_1 = './imagenes/img_cal_set1'
CALIBRATION_SET_2 = './imagenes/img_cal_set2'

# Constants

EXTRA_IMG = 'imgCalExtr'
IMG_NAME = 'imgBloque'

# Debug
IMGS_DEBUG = ['imgBloque1', 'imgBloque2', 'imgBloque3', 'imgBloque4', 'imgBloque16']

## Auxiliary Functions

In [ ]:
# Auxiliary function to make the plots
def plotter(image, title = '', imgSize = (16,9), grayScale = False):
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.show()

# Auxiliary function to load all the images from path
def loadImages(path):
    imgNames = glob(path + '/*')
    result = {}
    for imgName in imgNames:
        img = cv.imread(imgName)
        result[re.sub(r'^.*/([^.]*).*$', r'\1', imgName)] = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return result

# Creates a mask from the image and the specified position. Remember: in OpenCv the coorinates are (Y, X)
def createMask(img, position, lowerMultiplier = 6, upperMultiplier = 6):
    initialPoint, endPoint = position
    colorMean, colorStd = cv.meanStdDev(img[initialPoint[1]:endPoint[1], initialPoint[0]:endPoint[0], :])
    return cv.inRange(img, colorMean - colorStd * lowerMultiplier,  colorMean + colorStd * upperMultiplier)

# Applies a mask to the img. Returns a new image with the mask applied
def applyMask(img, mask):
    return cv.bit

# Plot the histogram for the first 'amountOfBlocks' blocks
def plotHistograms(imgs, bins = 50, amountOfBlocks = 5):
    fig, axs = plt.subplots(amountOfBlocks)
    fig.suptitle('Histograms', fontsize=18, fontweight='bold')
    imgsNames = [IMG_NAME + str(i) for i in range(1, amountOfBlocks + 1)]
    xTicks = np.arange(0, 260, 10)
    yTicks = np.arange(0, 300000, 30000)
    for imgNum, imgName in enumerate(imgsNames):
        axs[imgNum].set_title(imgName, fontsize = 16, fontweight='bold')
        axs[imgNum].grid()
        axs[imgNum].hist(imgs[imgName].ravel(),bins,[0,256], color='orange')
        axs[imgNum].set_xticks(xTicks)
        axs[imgNum].set_yticks(yTicks)
    
    fig.set_size_inches(16, 10)
    fig.tight_layout(pad=5.0)
    
# Performs the Otsu binarization mehtod. Returns a dictionary with the images binarized
def otsuBinarization(imgs, thresh = 100):
    result = {}
    for imgName, img in imgs.items():
        ret, imgBin = cv.threshold(img, thresh, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
        result[imgName] = imgBin
    return result

# For debugging
def debug(imgs, grayScale=False):
    for imgName in IMGS_DEBUG:
        plotter(imgs[imgName], imgName, grayScale=grayScale)

## Process Images

Primero cargo las imagenes de los bloques y las de los sets de calibracion.

## Hay que rotar todas las imagenes de calibracion para que apunten en la misma direccion??

In [ ]:
imgsBlocks = loadImages(BLOCKS_PATH)
calibrationSet1 = loadImages(CALIBRATION_SET_1)
calibrationSet2 = loadImages(CALIBRATION_SET_2)

# Dado que en las imagenes de los bloques hay una imagen para realizar la calibracion extrinseca
# la guardo en una variable y la elimino del set a analizar
extrinsicCalibration = imgsBlocks[EXTRA_IMG]
del imgsBlocks[EXTRA_IMG]

# Imprimo algunas imagenes para verificar que este todo bien
debug(imgsBlocks)

Creo una mascara con el tablero verde para poder eliminar el ruido de los otros elementos que se encuentran en la imagen. Para crearla, tomare una porcion de 'imgBloque1' y otra de 'imgBloque2' para luego hacer un OR y obtener una mascara que solo abarque el tablero verde.

In [ ]:
# Position = ((x1, y1), (x2, y2))
maskPosition1 = ((125, 280), (480, 450))
blocksMask1 = createMask(imgsBlocks['imgBloque1'], maskPosition1)

maskPosition2 = ((125, 50), (480, 250))
blocksMask2 = createMask(imgsBlocks['imgBloque4'], maskPosition2)
blocksMask = cv.bitwise_or(blocksMask1, blocksMask2)
blocksMask = cv.medianBlur(blocksMask, 15)

blocksWithMask = {}
for imgName, img in imgsBlocks.items():
    blocksWithMask[imgName] = cv.bitwise_and(img, img, mask=blocksMask)

# DEBUG
debug(blocksWithMask)

Ahora procedo a crear una mascara para los bloques para luego aplicarla y hacer que solo ellos se encuentren denifidos en las imagenes. Usare los bloques de las imagenes 1, 2 y 3 para obtener la mascara deseada.

In [ ]:
# Blocks Positions
block1 = ((325, 110), (400, 125))
block2 = ((285, 150), (325, 195))
block3 = ((400, 150), (450, 200))
blocks = [block1, block2, block3]

# Draw a red line to verify the positions
for blockNum, block in enumerate(blocks):
    testImg = blocksWithMask[IMG_NAME + str(blockNum + 1)].copy()
    cv.line(testImg, block[0], block[1], (255,0,0), 5)
    plotter(testImg, IMG_NAME + str(blockNum + 1))

# Get statistics about each block to compute the mask
totalMeanBlocks = 0
totalStdBlocks = 0
for blockNum, blockPosition in enumerate(blocks):
    initialPoint, endPoint = blockPosition
    imgName = IMG_NAME + str(blockNum + 1)
    blockImg = blocksWithMask[imgName][initialPoint[1]:endPoint[1], initialPoint[0]:endPoint[0], :]
    meanBlock, stdBlock = cv.meanStdDev(blockImg)
    totalMeanBlocks += meanBlock
    totalStdBlocks += stdBlock
    plotter(blockImg, imgName, imgSize=(8,5))

meanBlocks = totalMeanBlocks / len(blocks)
stdBlocks = totalStdBlocks / len(blocks)

In [ ]:
print(f"Mean: {meanBlocks} \nStd: {stdBlocks}")

Aplico estos datos obtenidos a todas las imagenes para obtnener solo los bloques definidos en cada una de ellas. Los resultados seran guardados en **blocksWithMask** en escala de grises, pisando los valores anteriores.

In [ ]:
for imgName, img in blocksWithMask.items():
    mask_i = cv.inRange(img, meanBlocks - stdBlocks * 6,  meanBlocks + stdBlocks * 6)
    blocksWithMask[imgName] = cv.cvtColor(cv.bitwise_and(img, img, mask=mask_i), cv.COLOR_RGB2GRAY)

debug(blocksWithMask, grayScale=True)

A continuacion grafico el histograma del los primeros dos bloques para tener una idea de la distribucion de los valores, asi establezco un threshold para luego poder binarizar las imagenes.

In [ ]:
plotHistograms(blocksWithMask, amountOfBlocks=2)

El resultado de los histogramas es el esperado, ya que el porcentaje de color del bloque es pequeño en comparación con toda la imagen que es negra. Usare un _threshold_ de 100 para realizar la binarizacion.

In [ ]:
binarizedBlocks = otsuBinarization(blocksWithMask)
debug(binarizedBlocks, grayScale=True)

Aplico operaciones morfologicas y filtros para corregir las imperfecciones de las imagenes.

In [ ]:
#PRUEBAS DE OPERACIONES MORFOLOGICAS
imgTest = binarizedBlocks['imgBloque1'].copy()

# Erode
kernelErode = np.ones((9,10), np.uint8)
plotter(cv.erode(imgTest, kernelErode), 'Erode', grayScale=True)

# Dilate
kernelDilate =np.ones((15,15), np.uint8)
plotter(cv.dilate(imgTest, kernelDilate), 'Dilate', grayScale=True)

# Opening
kernelOpening = np.ones((30,30), np.uint8)
plotter(cv.morphologyEx(imgTest, cv.MORPH_OPEN, kernelOpening),  'Opening', grayScale=True)

# Closing
kernelClosing = np.ones((30,30), np.uint8)
plotter(cv.morphologyEx(imgTest, cv.MORPH_CLOSE, kernelClosing), 'Closing', grayScale=True)

# Multiple Erosions and Dilations

kernelErotion2 = np.ones((15,15), np.uint8)
kernelDilation2 = np.ones((15,10), np.uint8)
imgTest2 = imgTest.copy()

imgTest2 = cv.erode(imgTest2, kernelErotion2, iterations=2)
imgTest2 = cv.dilate(imgTest2, kernelDilation2, iterations=5)
plotter(imgTest2, 'Multiple Operations', grayScale=True)

La que dio mejores resultados fue la operacion de _closing_. Utilizare esta en combinacion con un _filtro de mediana_ para todas las imagenes.

In [ ]:
# This dictionary will contain the final result of all the previous operation 
# plus the next one for the block images
KERNEL_SIZE = 5
blocksImgs = {}
for blockName, blockImg in binarizedBlocks.items():
    blockImg = cv.morphologyEx(blockImg, cv.MORPH_CLOSE, kernelClosing)
    blocksImgs[blockName] = cv.medianBlur(blockImg, KERNEL_SIZE)

debug(blocksImgs, grayScale=True)

# Prueba Auxiliar con tp7

In [ ]:
contours, hier = cv.findContours(blocksImgs['imgBloque16'], cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
out = imgsBlocks['imgBloque16'].copy()
cv.drawContours(out, contours, -1, (255,0,0),2)
plotter(out)

In [ ]:
def getAspectRatio(contour):
    x,y,width,height = cv.boundingRect(contour)
    return np.round(float(width) / height, 2)

def getCentroid(moment, area):
    return (int(moment['m10'] / area), int(moment['m01'] / area))

def getOrientation(contour):
    (x, y), (MA, mA), angle = cv.fitEllipse(contour)
    return (int(x), int(y)), (int(MA), int(mA)), np.round(angle)

In [ ]:
# Para mas info https://en.wikipedia.org/wiki/Image_moment
AREA = 0 #Para saber en que posicion del array que guarda el diccionario se encuentra la info deseada
PERIMETER = 1
CENTER = 2
ASPECT_RELATION = 3
ORIENTATION = 4
CONTOUR = 5

def getContoursInfo(contours):
    contoursArea = []
    contoursPerimeter = []
    contoursCenter = []

    for ctr in contours:
        moment = cv.moments(ctr)
        area = int(moment['m00'])
        contoursArea.append(area)
        contoursPerimeter.append(int(cv.arcLength(ctr,True)))
        contoursCenter.append(getCentroid(moment, area))

    contoursAreaMean = np.mean(contoursArea)

    contoursInfo = {}  
    contourNum = 1
    for area, perimeter, center, contour in  zip(contoursArea, contoursPerimeter, contoursCenter, contours):
        if area >= contoursAreaMean:
            contoursInfo[contourNum] = [area, perimeter, center, getAspectRatio(contour),
                                        getOrientation(contour), contour]
            contourNum += 1
            
    return contoursInfo

In [ ]:
contoursInfo = getContoursInfo(contours)

def printResults(contoursInfo):
    print('RESULTS')
    for key, value in contoursInfo.items():
        print('''
                Block Num: {}

                    Area: {}
                    Perimeter: {}
                    Centroid: {}
                    Aspect Relation: {}
                    Orientation: {}
                '''.format(key, value[AREA], value[PERIMETER], value[CENTER], value[ASPECT_RELATION],
                          value[ORIENTATION][2]))
    

In [ ]:
imgCut = imgsBlocks['imgBloque16']

In [ ]:
out = imgCut.copy()
font = cv.FONT_HERSHEY_SIMPLEX
for key, value in contoursInfo.items():
    contour = value[CONTOUR]
    cv.putText(out, str(key), value[CENTER], font, 2, (0, 0, 0), 3, cv.LINE_AA)
    cv.circle(out, value[CENTER], radius=5, color=(255, 255, 255), thickness=-3)
    approx = cv.approxPolyDP(contour, 0.1 * cv.arcLength(contour, True), True)
    cv.drawContours(out, [approx], 0, (255, 0, 0), 2)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
plotter(out)

In [ ]:
# Dibujamos las elipese que se obtuvieron al calcular la orientacion mas arriba
out = imgCut.copy()
font = cv.FONT_HERSHEY_SIMPLEX
for key, value in contoursInfo.items():
    contour = value[CONTOUR]
    center, axes, angle = value[ORIENTATION]
    ellipse = cv.fitEllipse(contour)
    cv.ellipse(out, ellipse, (0,0,0), 2)
    cv.putText(out, str(key), value[CENTER], font, 2, (0, 0, 0), 3, cv.LINE_AA)
    approx = cv.approxPolyDP(contour, 0.1 * cv.arcLength(contour, True), True) 
    cv.drawContours(out, [approx], 0, (255, 0, 0), 2)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
plotter(out)

In [ ]:
printResults(contoursInfo)

## Step 1 - Intrinsic Parameters

## Step 2 - Extrinsic Calibration

## Step 3 - Search Blocks

## Step 4 - Validation

## Step 5 - Measure Blocks